In [ ]:
# Pandas, azure-storage-blob, dotenv

%pip install pandas azure-storage-blob
%pip install python-dotenv

In [2]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os

In [3]:
try:
    data = pd.read_csv(r'dataset/bakerite_transaction.csv')
    print('Data Extraction Successful')
except Exception as e:
    print(f"An error occurred {e}") 

Data Extraction Successful


In [4]:
# Fill missing numeric values with the mean or median.

numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_columns:
    data.fillna({col: data[col].mean()}, inplace=True)  

In [5]:
# Fill missing string/object with 'Unknown'

string_columns = data.select_dtypes(include=['object']).columns
for col in string_columns:
    data.fillna({col: 'Unknown'}, inplace=True)

In [6]:
data['Date'] = pd.to_datetime(data['Date']) 

In [7]:
data.columns

Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')

In [8]:
# Products Table

products = data[['ProductName']].drop_duplicates().reset_index(drop=True)
products.index.name = 'ProductID'
products = products.reset_index()

In [ ]:
# Customers Table

customers = data[['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail']].drop_duplicates().reset_index(drop=True)
customers.index.name = 'CustomerID'
customers = customers.reset_index()

In [ ]:
# Staff Table

staff = data[['Staff_Name', 'Staff_Email']].drop_duplicates().reset_index(drop=True)
staff.index.name = 'StaffID' 
staff = staff.reset_index()

In [11]:
transaction = data.merge(products, on=['ProductName'], how='left') \
                  .merge(customers, on=['CustomerName', 'CustomerAddress',      'Customer_PhoneNumber', 'CustomerEmail'], how='left') \
                  .merge(staff, on=['Staff_Name', 'Staff_Email'], how='left')

transaction.index.name = 'TransactionID'
transaction = transaction.reset_index() \
                        [['Date', 'TransactionID', 'ProductID', 'Quantity', 'UnitPrice', 'StoreLocation', 'PaymentType', 'PromotionApplied', 'Weather', 'Temperature','StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min', 'OrderType', 'CustomerID', 'StaffID', 'DayOfWeek', 'TotalSales']]

In [12]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1005 non-null   datetime64[ns]
 1   TransactionID           1005 non-null   int64         
 2   ProductID               1005 non-null   int64         
 3   Quantity                1005 non-null   int64         
 4   UnitPrice               1005 non-null   float64       
 5   StoreLocation           1005 non-null   object        
 6   PaymentType             1005 non-null   object        
 7   PromotionApplied        1005 non-null   bool          
 8   Weather                 1005 non-null   object        
 9   Temperature             1005 non-null   float64       
 10  StaffPerformanceRating  1005 non-null   object        
 11  CustomerFeedback        1005 non-null   object        
 12  DeliveryTime_min        1005 non-null   int64   

In [ ]:
data.to_csv('clean_data.csv', index=False)
products.to_csv('products.csv', index=False)
customers.to_csv('customers.csv', index=False)
staff.to_csv('staff.csv', index=False)
transaction.to_csv('transactions.csv', index=False)

In [ ]:
# Load variables from .env

load_dotenv()

connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
container_name = os.getenv('AZURE_CONTAINER_NAME')

In [ ]:
# Create a BlobServiceClient

blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

# Loading data to Azure Blob Storage
files = [
    (data, 'rawdata/cleaned_bakerite_transaction_data.csv'),
    (products, 'cleandata/products.csv'),
    (customers, 'cleandata/customers.csv'),
    (staff, 'cleandata/staff.csv'),
    (transaction, 'cleandata/transaction.csv')
]

for file, blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f'{blob_name} loaded into Azure Blob Storage')

rawdata/cleaned_bakerite_transaction_data.csv loaded into Azure Blob Storage
cleandata/products.csv loaded into Azure Blob Storage
cleandata/customers.csv loaded into Azure Blob Storage
cleandata/staff.csv loaded into Azure Blob Storage
cleandata/transaction.csv loaded into Azure Blob Storage
